# Problem 4.1

**Integrated Energy Grids**

**Problem 4.1**

**This is a continuation of Problem 3.2 from Lecture 3:
Let us assume now that we are in an hour with an excess of wind generation in Denmark and a deficit in other countries so that the power injection $P_i$ of the different countries is as follows: Germany= - 200 MW, DK1=500 MW, DK2=600 MW, Norway = - 800 MW, Sweden = -100 MW.**

**Determine the voltage angles $\theta_i$  and the flows $P_l$ in the lines of the network. Assume that $\theta_0$=0; i.e. the reference bus is at node 0 (Germany); and the reactance in all links are $x_l$=1.**


We will use the package [numpy](https://numpy.org/) to operate with matrices.

In [49]:
import numpy as np
import numpy.linalg

Calculate list of nodes, links, degree, adjacency and Laplacian matrix. (this was already implemented in Problem 3.2)

In [50]:
nodes=[0,1,2,3,4]
links=[(0,1), (1,2), (1,3), (1,4), (2,4)]

In [51]:
D = np.zeros((len(nodes), len(nodes)))

for node in nodes:
    D[node, node] = sum([1 if node in link else 0 for link in links])

In [52]:
A = np.zeros((len(nodes), len(nodes)))

for node_a, node_b in links:
    A[node_a, node_b] = 1
    A[node_b, node_a] = 1


In [53]:
L = D - A
L

array([[ 1., -1.,  0.,  0.,  0.],
       [-1.,  4., -1., -1., -1.],
       [ 0., -1.,  2.,  0., -1.],
       [ 0., -1.,  0.,  1.,  0.],
       [ 0., -1., -1.,  0.,  2.]])

In [54]:
K = np.zeros((len(nodes),len(links)))

for i, (node_a, node_b) in enumerate(links):
    K[node_a,i] = 1
    K[node_b,i] = -1
    
K

array([[ 1.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  1.,  0.],
       [ 0., -1.,  0.,  0.,  1.],
       [ 0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0., -1., -1.]])

**Power flow analysis**

We know the power injection pattern for the nodes $P_i$ and we want to determine the power flows $P_l$ in the lines of the network



The reactance in every link is $x_l$=1 so $L_{ij}=\sum_{l} K_{il} \frac{1}{x_l}K_{lj}=\sum_{l} K_{il} K_{lj}$ or  $L_{ij}=D_{ij}-A_{i,j}$

First, we calculate the voltage angles, by solving the set of linear equations.
$P_i=\sum L_{i,j} \theta_j$


In [55]:
P_i =[-200, 500, 600, -800, -100]

The node 0 (Germany) is the slack bus, so $\theta_0$=0.

In [56]:
theta = np.r_[0, np.linalg.solve(L[1:,1:], P_i[1:])]
theta

array([   0.        ,  200.        ,  566.66666667, -600.        ,
        333.33333333])

Second, we use the voltage angles to compute the flows.

$P_l=\frac{1}{x_l}\sum_{j}K_{lj}\theta_j$

In [57]:
P_l= K.T.dot(theta)
P_l

array([-200.        , -366.66666667,  800.        , -133.33333333,
        233.33333333])

**Additional discussion on the inversion of the Laplacian matrix**

The Laplacian matrix is not invertible because it has one zero eigenvalue with eigenvector (1,1,1,1,1). 

We can only invert a matrix if it is non-singular, i.e., the determinant should not be zero. 

You can also check that the rank of the matrix is 4 but the matrix has 5 columns, so the determinant is zero and we can not find the inverse


In [58]:
np.linalg.matrix_rank(L)

4

Instead of selecting Germany as the slack bus, we could have used the [Moore Penrose pseudo-inverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse) to invert the Laplacian. 

In this case, we don't get bus zero as slack and the theta values in the solution are shifted with respect to the previous solution, but we get the same results for the power flows.  

In [59]:
theta_ = np.linalg.pinv(L).dot(P_i)
theta_

array([-100.        ,  100.        ,  466.66666667, -700.        ,
        233.33333333])

In [61]:
P_l= K.T.dot(theta_)
P_l

array([-200.        , -366.66666667,  800.        , -133.33333333,
        233.33333333])

**Additional discussion on the magnitude of theta**

For the sake of simplicity, in this problem, we are indicating the power injection in MW (instead of in per unit) which causes $\theta$ not to be expressed in radians.